In [1]:
import pandas as pd 
import numpy as np

## Rec2TF ##

In [65]:
TF_interactions = pd.read_csv('./Receptor2TF.csv')
LR_interactions = pd.read_csv('./LR_interactions.csv')

In [66]:
RecTF = pd.read_table(r'D:\DLCCC\LR_Source\scMLnet\RecTF.txt')

In [67]:
RecTF.columns = ['receptor', 'TF', 'label']

In [68]:
RecTF2 = pd.read_table(r'D:\DLCCC\LR_Source\cellcall\new_ligand_receptor_TFs.txt')


In [69]:
RecTF2 = RecTF2[['Ligand_Symbol', 'Receptor_Symbol', 'TF_Symbol']]

In [70]:
RecTF2.columns = ['ligand', 'receptor', 'TF']

In [71]:
RecTF['ligand'] = 'NA'

In [72]:
RecTF = pd.concat([RecTF[['ligand', 'receptor', 'TF']], RecTF2])

In [73]:
TF_interactions_cpdb = TF_interactions.loc[TF_interactions['Source'] == 'CellPhoneDB']
TF_interactions_cpdb.columns = ['receptor', 'TF', 'Source']
TF_interactions_cpdb['ligand'] = 'NA'
RecTF = pd.concat([RecTF, TF_interactions_cpdb[['receptor', 'TF']]])

C:\Users\13175\AppData\Local\Temp\ipykernel_33856\1861979643.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TF_interactions_cpdb['ligand'] = 'NA'


In [74]:
RecTF.drop_duplicates(subset=['receptor', 'TF'], inplace=True)

In [75]:
RecTF

,ligand,receptor,TF
0,NA,CFTR,NR3C2
1,NA,CFTR,JUND
2,NA,CFTR,MAF
3,NA,CFTR,NR3C1
4,NA,CFTR,GATA1
...,...,...,...
41423,NaN,IL6,STAT1
41424,NaN,IL7,STAT5A
41425,NaN,IFNGR1&IFNGR2,STAT2
41426,NaN,IL10RB&IFNLR1,STAT1


In [76]:
print(RecTF['receptor'].unique().__len__())
print(RecTF['TF'].unique().__len__())

769
1019


## TF2Target ##

In [77]:
TFTgt = pd.read_table(r'D:\DLCCC\LR_Source\scMLnet\TFTargetGene.txt')
TFTgt_2 = pd.read_table(r'D:\DLCCC\LR_Source\cellcall\tf_target.txt')

In [78]:
TFTgt_2 = TFTgt_2[['TF_Symbol', 'Target_Symbol']]
TFTgt = TFTgt[['TF', 'Targets']]

In [79]:
TFTgt.columns = ['TF', 'Targets']
TFTgt_2.columns = ['TF', 'Targets']

In [80]:
TFTgt['Source'] = 'scMLnet'
TFTgt_2['Source'] = 'cellcall'

In [81]:
TFTgt = pd.concat([TFTgt, TFTgt_2])

In [82]:
TFTgt = TFTgt.drop_duplicates(subset=['TF', 'Targets'])

In [83]:
TFTgt

,TF,Targets,Source
0,AATF,BAX,scMLnet
1,AATF,CDKN1A,scMLnet
2,AATF,KLK3,scMLnet
3,AATF,MYC,scMLnet
4,AATF,TP53,scMLnet
...,...,...,...
591010,TP53,WEE1,cellcall
591011,TP53,WTAP,cellcall
591012,TP53,ZIM2,cellcall
591013,TP53,ZNF266,cellcall


In [84]:
print(TFTgt['TF'].unique().__len__())
print(TFTgt['Targets'].unique().__len__())

856
18502


In [86]:
## Merge Receptor-TF and TF-TargetGene
print((set(RecTF['TF'].unique()) & set(TFTgt['TF'].unique())).__len__())

834


In [85]:
TF_intersections = set(RecTF['TF'].unique()) & set(TFTgt['TF'].unique())

In [87]:
RecTF = RecTF.loc[RecTF['TF'].isin(TF_intersections)]

In [88]:
TFTgt = TFTgt.loc[TFTgt['TF'].isin(TF_intersections)]

In [89]:
print((set(RecTF['TF'].unique()) & set(TFTgt['TF'].unique())).__len__() / len(RecTF['TF'].unique()))

1.0


In [90]:
print(RecTF.shape)
print(TFTgt.shape)

(40699, 3)
(595619, 3)


In [91]:
RecTF.to_csv('./LR_TF_Tgt/Receptor2TF.csv', index=False)
TFTgt.to_csv('./LR_TF_Tgt/TF2TargetGene.csv', index=False)

In [92]:
print(RecTF['receptor'].unique().__len__())
print(RecTF['TF'].unique().__len__())
print(TFTgt['TF'].unique().__len__())
print(TFTgt['Targets'].unique().__len__())

766
834
834
18501


In [94]:
ligands = LR_interactions['Src'].unique().tolist()
receptors = LR_interactions['Dst'].unique().tolist()    

In [96]:
len((set(TFTgt['Targets']) & set(receptors))) / len(receptors)

0.8554259455638035

In [97]:
len((set(TFTgt['Targets']) & set(ligands))) / len(ligands)

0.8823281315900464

In [ ]:
interactions_path = '../LR_Base/LR_interactions.csv'
complex_nodes_path = '../LR_Base/complexes_df.csv'
LR_interactions = pd.read_csv(interactions_path)
complex_nodes = pd.read_csv(complex_nodes_path)
LR_nodes = list(set(LR_interactions['Src'].tolist() + LR_interactions['Dst'].tolist() + complex_nodes['subunit'].tolist()))


# only LR interactions

cell_nodes_exp = get_gene_expression(adata, LR_nodes=LR_nodes, deg=deg, prop=prop, celltype=args.celltype)
complex_expr_df = calculate_complex_expr(complex_nodes, cell_nodes_exp)

ligands = list(set(LR_interactions['Src']))
receptors = list(set(LR_interactions['Dst']))
ligand_exp = get_ligand_exp(cell_nodes_exp, ligands)
receptor_exp = get_receptor_exp(cell_nodes_exp, receptors)
ligand_exp, receptor_exp = merge_single_complex(ligand_exp, receptor_exp, complex_expr_df)

filtered_LR_interactions, ligand_exp, receptor_exp = filter_LR_interactions(LR_interactions, ligand_exp, receptor_exp)
LR_interactions_expression = get_LR_interactions_expression(filtered_LR_interactions, ligand_exp, receptor_exp)

id_dict = create_id_dict(cell_nodes_exp, ligand_exp, receptor_exp, None)

LR_interactions_expression = get_LR_interactions_id(LR_interactions_expression, id_dict)
if args.spatial:
    spatial_interactions = get_spatial_interactions(adata, id_dict, args.spatial_cutoff, args.celltype)
    data = create_hetero_data(LR_interactions_expression, None, spatial_interactions, id_dict)
else:
    data = create_hetero_data(LR_interactions_expression, None, None, id_dict)

In [ ]:
filtered_rec2tf = filtered_rec2tf.loc[(filtered_rec2tf['receptor'].isin(cell_nodes_exp['gene'])) & (filtered_rec2tf['TF'].isin(cell_nodes_exp['gene']))]
filtered_tf2tgt = filtered_tf2tgt.loc[(filtered_tf2tgt['TF'].isin(cell_nodes_exp['gene'])) & (filtered_tf2tgt['Targets'].isin(cell_nodes_exp['gene']))]

## LR ##

In [55]:
LR_interactions = pd.read_csv('LR_complexes.csv')

In [92]:
LR_interactions.to_csv('new_LR_interactions.csv', index=False)

In [91]:
LR_interactions

,Src,Dst
0,5HT,HTR1A
1,5HT,HTR1B
2,5HT,HTR1D
3,5HT,HTR1E
4,5HT,HTR1F
...,...,...
38453,IL1B,NR2C2
38454,FYN,GRM1
38455,AR,TRHR
38456,PTK2,CNR1


In [90]:
LR_interactions = LR_interactions.drop_duplicates(['Src', 'Dst'])
LR_interactions = LR_interactions.dropna()

In [69]:
nichenet_df = pd.read_csv('./Source/NicheNet_df.csv', index_col=0)

In [71]:
nichenet_df['database'].unique()

array(['kegg', 'guide2pharmacology', 'ramilowski', 'ppi_prediction',
       'ppi_prediction_go'], dtype=object)

In [73]:
filter_nichenet = nichenet_df[nichenet_df['database'].isin(keywords)]

In [72]:
keywords = ['ppi_prediction', 'ppi_prediction_go']

In [75]:
filter_nichenet = filter_nichenet[['Ligand_Symbol', 'Receptor_Symbol']]

In [77]:
filter_nichenet.columns = ['Src', 'Dst']
filter_nichenet

,Src,Dst
1980,CCL5,SDC4
1981,EDN2,EDNRB
1982,CSF2,SDC2
1983,LAMA1,GPC1
1984,TDGF1,GPC1
...,...,...
12655,CXCL6,PIGR
12656,RLN1,GPR25
12657,RLN1,RAMP3
12658,PIK3CB,TNFRSF10A


In [78]:
filter_nichenet = filter_nichenet.drop_duplicates(['Src', 'Dst'])
filter_nichenet = filter_nichenet.dropna()

In [79]:
filter_nichenet

,Src,Dst
1980,CCL5,SDC4
1981,EDN2,EDNRB
1982,CSF2,SDC2
1983,LAMA1,GPC1
1984,TDGF1,GPC1
...,...,...
12655,CXCL6,PIGR
12656,RLN1,GPR25
12657,RLN1,RAMP3
12658,PIK3CB,TNFRSF10A


In [80]:
A = LR_interactions
B = filter_nichenet

In [81]:
# 找到A和B的交集
intersection = pd.merge(A, B, how='inner')


In [82]:
intersection

,Src,Dst
0,ADAM15,CDH1
1,ADAM17,GHR
2,ADAM17,IL1R2
3,ADAM17,IL6R
4,ADAM17,NGFR
...,...,...
3099,YARS,RPSA
3100,YARS,SORT1
3101,YARS,VCAM1
3102,ZP3,PCDH17


In [83]:
# 将交集从A和B中删除
LR_interactions = pd.concat([A, intersection]).drop_duplicates(keep=False)

In [85]:
LR_interactions.index = range(len(LR_interactions))

In [86]:
LR_interactions

,Src,Dst
0,5HT,HTR1A
1,5HT,HTR1B
2,5HT,HTR1D
3,5HT,HTR1E
4,5HT,HTR1F
...,...,...
38453,IL1B,NR2C2
38454,FYN,GRM1
38455,AR,TRHR
38456,PTK2,CNR1


## Complex ##

In [1]:
import pandas as pd

In [2]:
com_interactions = pd.read_csv('./LR_complexes.csv')

In [3]:
com_interactions

,ligand,receptor,Source
0,IFNA1,IFNAR1&IFNAR2,cellcall
1,IFNA2,IFNAR1&IFNAR2,cellcall
2,IFNA4,IFNAR1&IFNAR2,cellcall
3,IFNA5,IFNAR1&IFNAR2,cellcall
4,IFNA6,IFNAR1&IFNAR2,cellcall
...,...,...,...
4341,CSH1&CSH2,PRLR,scconnect
4342,TGFB1,TGFBR1&TGFBR2&TGFBR3,scconnect
4343,TGFB2,TGFBR1&TGFBR2&TGFBR3,scconnect
4344,TGFB3,TGFBR1&TGFBR2&TGFBR3,scconnect


In [8]:
complexes = com_interactions['ligand'][com_interactions['ligand'].str.contains('&')].tolist() + com_interactions['receptor'][com_interactions['receptor'].str.contains('&')].tolist()

In [13]:
complexes = list(set(complexes))

In [14]:
set(complexes).__len__()

646

In [15]:
complex_df = pd.DataFrame(complexes, columns=['complex'])

In [4]:
Src_list = com_interactions['ligand'].str.split('&').explode().tolist()
Dst_list = com_interactions['receptor'].str.split('&').explode().tolist()

In [24]:
complex_df.index = complex_df['complex'].tolist()

In [31]:
complex_df

,subunit,complex
0,ACVRL1,ACVRL1&ACVR2A
1,ACVR2A,ACVRL1&ACVR2A
2,AMHR2,AMHR2&ACVR1
3,ACVR1,AMHR2&ACVR1
4,TGFBR1,TGFBR1&TGFBR2&TGFBR3
...,...,...
1421,IL1B,IL1B&IL1R2&IL1RAP
1422,IL1R2,IL1B&IL1R2&IL1RAP
1423,IL1RAP,IL1B&IL1R2&IL1RAP
1424,ITGAM,ITGAM&ITGB2


In [27]:
subnits_df = complex_df['complex'].apply(lambda x: pd.Series(x.split('&'))).stack().reset_index(level=1, drop=True)

In [32]:
complex_df.to_csv('complexes_df.csv', index=False)

In [28]:
subnits_df = subnits_df.reset_index().rename(columns={0: 'subunit', 'index': 'complex'})

In [30]:
complex_df = subnits_df[['subunit', 'complex']]

## calculate complex expression ##

In [1]:
from utils import *

In [2]:
path = '../input/Visium_RZ_FZ_P5.h5ad'
interactions_path = '../LR_Base/LR_single.csv'
complex_nodes_path = '../LR_Base/complexes_df.csv'
complex_interactions_path = '../LR_Base/LR_complexes.csv'

In [ ]:
cell_LR_nodes, cell_LR_interactions, complex_expr_df, meta, celltype  = load_data(path, interactions_path, complex_nodes_path, cutoff=0.9)

In [ ]:
cell_LR_nodes

,gene,expression,identifier,category,cellgroup
CCL14,CCL14,0.198610,CCL14_0_Ligand,Ligand,0
CCL14,CCL14,0.402186,CCL14_1_Ligand,Ligand,1
CCL14,CCL14,0.437958,CCL14_2_Ligand,Ligand,2
CCL14,CCL14,0.704263,CCL14_3_Ligand,Ligand,3
CCL14,CCL14,0.338172,CCL14_4_Ligand,Ligand,4
...,...,...,...,...,...
RBP1,RBP1,0.080605,RBP1_0_Receptor,Receptor,0
RBP1,RBP1,0.141357,RBP1_1_Receptor,Receptor,1
RBP1,RBP1,0.023211,RBP1_2_Receptor,Receptor,2
RBP1,RBP1,0.066893,RBP1_3_Receptor,Receptor,3


In [16]:
valid_complexes_dict = filter_complex_nodes(cell_LR_nodes, complex_nodes_path)

In [18]:
for i in valid_complexes_dict.keys():
    print(i, valid_complexes_dict[i].__len__())

0 82
1 82
2 76
3 82
4 30
5 14
6 5


In [17]:
valid_complexes_dict

{'0': ['ACVR2A&BMPR1B',
  'ACVR2A&TGFBR1',
  'B2M&FCGRT',
  'BMPR1B&ACVR2A',
  'BMPR1B&BMPR2',
  'BMPR2&TGFBR1',
  'BOC&PTCH1',
  'BSG&SLC16A1',
  'CALM2&CALM3&CALM1',
  'CALR&LRP1',
  'CD44&CD74',
  'CD63&ITGB1',
  'CD74&CD44',
  'CD74&CXCR2',
  'CD74&CXCR4',
  'CDC37&HSP90AA1',
  'EZR&L1CAM',
  'FGFR1&KL',
  'FGFR1&TGFBR3',
  'FLT1&KDR',
  'FZD2&LRP5',
  'FZD2&LRP6',
  'FZD6&LRP5',
  'FZD6&LRP6',
  'FZD8&LRP5',
  'FZD8&LRP6',
  'FZD8&RYK',
  'GFRA3&RET',
  'IFNGR1&IFNGR2',
  'IFNGR2&IFNGR1',
  'IL1B&IL1R1&IL1RAP',
  'IL1R1&IL1RAP',
  'IL1RAP&IL1R1',
  'IL1RAP&IL1RL1',
  'IL1RL1&IL1RAP',
  'IL27RA&IL6ST',
  'IL6ST&IL27RA',
  'IL6ST&LIFR',
  'ITGA1&ITGB1',
  'ITGA2&ITGB1',
  'ITGA5&ITGB1',
  'ITGA6&ITGB1',
  'ITGA7&ITGB1',
  'ITGA8&ITGB1',
  'ITGAV&ITGB1',
  'ITGAV&ITGB3',
  'ITGAV&ITGB5',
  'ITGB1&ITGA1',
  'ITGB1&ITGA2',
  'ITGB1&ITGA5',
  'ITGB1&ITGA6',
  'ITGB1&ITGA7',
  'ITGB1&ITGA8',
  'ITGB1&ITGAV',
  'ITGB3&ITGAV',
  'ITGB5&ITGAV',
  'LIFR&IL6ST',
  'LRP5&FZD2',
  'LRP5&FZD6',


In [19]:
complex_nodes = pd.read_csv(complex_nodes_path)

In [ ]:
complex_expr_df = calculate_complex_expression(cell_LR_nodes, valid_complexes_dict, complex_nodes)

In [ ]:
def filter_complex_nodes(cell_LR_nodes, complex_nodes_path):
    complex_nodes = pd.read_csv(complex_nodes_path)
    complex_dict = complex_nodes.groupby('complex')['subnit'].apply(list).to_dict()
    valid_complexes_dict = {}
    for cellgroup in cell_LR_nodes['cellgroup'].unique():
        cell_LR_nodes_cellgroup = cell_LR_nodes[cell_LR_nodes['cellgroup'] == cellgroup]
        cell_genes = cell_LR_nodes_cellgroup['gene'].unique().tolist()
        filter_complex_nodes = complex_nodes[complex_nodes['subnit'].isin(cell_genes)]
        filter_complex_dict = filter_complex_nodes.groupby('complex')['subnit'].apply(list).to_dict()
        valid_complexes = []
        for complex in filter_complex_dict.keys():
            if complex_dict[complex] == filter_complex_dict[complex]:
                valid_complexes.append(complex)
        valid_complexes_dict[cellgroup] = valid_complexes
    return valid_complexes_dict


def get_complex_interactions(cell_LR_nodes, cell_complex_df, complex_interactions, complex_interactions_single):
    cell_complex_single = cell_LR_nodes.loc[cell_LR_nodes['gene'].isin(complex_interactions_single['gene'])]
    cell_complex_nodes = pd.concat([cell_complex_df, cell_complex_single], axis=0)
    cell_complex_ligand = cell_complex_nodes.loc[cell_complex_nodes['category'] == 'Ligand']
    cell_complex_receptor = cell_complex_nodes.loc[cell_complex_nodes['category'] == 'Receptor']
    cell_complex_interactions = complex_interactions.loc[complex_interactions['Source'].isin(cell_complex_ligand['gene'])]
    cell_complex_interactions = cell_complex_interactions.loc[cell_complex_interactions['Target'].isin(cell_complex_receptor['gene'])]
    return cell_complex_interactions

In [58]:
complex_expr = {}
for cellgroup in cell_LR_nodes['cellgroup'].unique():
    complex_expr[cellgroup] = {}
    cell_LR_nodes_cellgroup = cell_LR_nodes[cell_LR_nodes['cellgroup'] == cellgroup]
    for i in valid_complexes_dict[cellgroup]:
        complex_expr[cellgroup][i] = []
        for j in i.split('&'):
            complex_expr[cellgroup][i].append(cell_LR_nodes_cellgroup[cell_LR_nodes_cellgroup['gene'] == j]['expression'].values[0])

In [59]:
complex_expr_df = []
for i in complex_expr.keys():
    for j in complex_expr[i].keys():
        complex_expr_df.append([j, i, np.min(complex_expr[i][j])])
complex_expr_df = pd.DataFrame(complex_expr_df, columns=['gene', 'cellgroup', 'expression'])


In [71]:
complex_df

,complex,category
ITGA4&ITGB1,ITGA4&ITGB1,Ligand
ITGA9&ITGB1,ITGA9&ITGB1,Ligand
ITGA4&ITGB7,ITGA4&ITGB7,Ligand
CRLF1&CLCF1,CRLF1&CLCF1,Ligand
IGHG1&IGLC1,IGHG1&IGLC1,Ligand
...,...,...
IL2RG&IL21R,IL2RG&IL21R,Receptor
IL6ST&IL27RA,IL6ST&IL27RA,Receptor
IL2RG&IL2RA&IL2RB,IL2RG&IL2RA&IL2RB,Receptor
IL6ST&IL6R,IL6ST&IL6R,Receptor


In [72]:
complex_expr_df['category'] = complex_expr_df['gene'].apply(lambda x: complex_df.loc[complex_df['complex'] == x]['category'].values[0])

In [84]:
complex_expr_df

,gene,cellgroup,expression,category,identifier
0,ACVR2A&BMPR1B,0,0.012901,Receptor,ACVR2A&BMPR1B_0_Receptor
1,ACVR2A&TGFBR1,0,0.070775,Receptor,ACVR2A&TGFBR1_0_Receptor
2,B2M&FCGRT,0,0.555856,Receptor,B2M&FCGRT_0_Receptor
3,BMPR1B&ACVR2A,0,0.012901,Receptor,BMPR1B&ACVR2A_0_Receptor
4,BMPR1B&BMPR2,0,0.012901,Ligand,BMPR1B&BMPR2_0_Ligand
...,...,...,...,...,...
366,B2M&FCGRT,6,2.222969,Receptor,B2M&FCGRT_6_Receptor
367,ITGAV&ITGB1,6,2.222969,Ligand,ITGAV&ITGB1_6_Ligand
368,ITGAV&ITGB5,6,2.222969,Ligand,ITGAV&ITGB5_6_Ligand
369,ITGB1&ITGAV,6,2.222969,Ligand,ITGB1&ITGAV_6_Ligand


In [88]:
cell_LR_nodes.index = range(cell_LR_nodes.shape[0])

In [85]:
cell_LR_nodes = pd.concat([cell_LR_nodes, complex_expr_df])

In [79]:
complex_nodes['category'] = complex_nodes['complex'].apply(lambda x: complex_df.loc[complex_df['complex'] == x]['category'].values[0])

In [81]:
category_complex = []
for i in complex_expr_df.index:
    category_complex.append(complex_nodes.loc[complex_nodes['complex'] == complex_expr_df.iloc[i]['gene']]['category'].values[0])
complex_expr_df['category'] = category_complex
complex_expr_df['identifier'] = complex_expr_df['gene'] + '_' + complex_expr_df['cellgroup'] + '_' + complex_expr_df['category']
complex_expr_df

,gene,cellgroup,expression,category,identifier
0,ACVR2A&BMPR1B,0,0.012901,Receptor,ACVR2A&BMPR1B_0_Receptor
1,ACVR2A&TGFBR1,0,0.070775,Receptor,ACVR2A&TGFBR1_0_Receptor
2,B2M&FCGRT,0,0.555856,Receptor,B2M&FCGRT_0_Receptor
3,BMPR1B&ACVR2A,0,0.012901,Receptor,BMPR1B&ACVR2A_0_Receptor
4,BMPR1B&BMPR2,0,0.012901,Ligand,BMPR1B&BMPR2_0_Ligand
...,...,...,...,...,...
366,B2M&FCGRT,6,2.222969,Receptor,B2M&FCGRT_6_Receptor
367,ITGAV&ITGB1,6,2.222969,Ligand,ITGAV&ITGB1_6_Ligand
368,ITGAV&ITGB5,6,2.222969,Ligand,ITGAV&ITGB5_6_Ligand
369,ITGB1&ITGAV,6,2.222969,Ligand,ITGB1&ITGAV_6_Ligand


In [82]:
complex_expr_df

,gene,cellgroup,expression,category,identifier
0,ACVR2A&BMPR1B,0,0.012901,Receptor,ACVR2A&BMPR1B_0_Receptor
1,ACVR2A&TGFBR1,0,0.070775,Receptor,ACVR2A&TGFBR1_0_Receptor
2,B2M&FCGRT,0,0.555856,Receptor,B2M&FCGRT_0_Receptor
3,BMPR1B&ACVR2A,0,0.012901,Receptor,BMPR1B&ACVR2A_0_Receptor
4,BMPR1B&BMPR2,0,0.012901,Ligand,BMPR1B&BMPR2_0_Ligand
...,...,...,...,...,...
366,B2M&FCGRT,6,2.222969,Receptor,B2M&FCGRT_6_Receptor
367,ITGAV&ITGB1,6,2.222969,Ligand,ITGAV&ITGB1_6_Ligand
368,ITGAV&ITGB5,6,2.222969,Ligand,ITGAV&ITGB5_6_Ligand
369,ITGB1&ITGAV,6,2.222969,Ligand,ITGB1&ITGAV_6_Ligand


In [4]:
LR_complexes = pd.read_csv(complex_interactions_path)

In [5]:
complexes = LR_complexes['Src'][LR_complexes['Src'].str.contains('&')].unique().tolist() 

In [9]:
complexes = LR_complexes['Dst'][LR_complexes['Dst'].str.contains('&')].unique().tolist()

In [10]:
complexes = list(set(complexes))

In [7]:
complex_df = pd.DataFrame(complexes, columns=['complex'])

In [8]:
complex_df['category'] = 'Ligand'

In [11]:
complex_df = pd.concat([complex_df, pd.DataFrame({'complex': complexes, 'category':'Receptor'})])

In [27]:
complex_df.loc[complex_df['complex'] == 'ITGAV&ITGB1', 'category']

ITGAV&ITGB1      Ligand
ITGAV&ITGB1    Receptor
Name: category, dtype: object

In [17]:
complex_df.index = complex_df['complex'].tolist()

In [28]:
complex_df_sub = complex_df['complex'].apply(lambda x: pd.Series(x.split('&'))).stack().reset_index(level=1, drop=True)

In [37]:
complex_df_sub.loc[complex_df_sub['complex'] == 'ITGAV&ITGB1']

,complex,subunit,category
263,ITGAV&ITGB1,ITGAV,Ligand
263,ITGAV&ITGB1,ITGAV,Receptor
264,ITGAV&ITGB1,ITGB1,Ligand
264,ITGAV&ITGB1,ITGB1,Receptor


In [30]:
complex_df_sub = complex_df_sub.reset_index().rename(columns={0: 'subunit', 'index': 'complex'})

In [35]:
complex_df_sub = complex_df_sub.drop_duplicates(subset=['complex', 'subunit'])

In [39]:
complex_df_sub.to_csv('complexes_df.csv', index=False)

In [36]:
complex_df_sub['category'] = complex_df_sub['complex'].apply(lambda x: list(complex_df.loc[complex_df['complex'] == x]['category'].values))
complex_df_sub = complex_df_sub.explode('category')

In [38]:
complex_df_sub

,complex,subunit,category
0,TENM3&FLRT3,TENM3,Ligand
1,TENM3&FLRT3,FLRT3,Ligand
2,ITGA4&ITGB7,ITGA4,Ligand
2,ITGA4&ITGB7,ITGA4,Receptor
3,ITGA4&ITGB7,ITGB7,Ligand
...,...,...,...
1480,ACVR1C&ACVR2A,ACVR2A,Receptor
1481,CSF2RA&CSF2RB,CSF2RA,Receptor
1482,CSF2RA&CSF2RB,CSF2RB,Receptor
1487,EGFR&ERBB2,EGFR,Receptor


In [22]:
list(complex_df.loc[complex_df['complex'] == 'ITGAV&ITGB1']['category'].values)

['Ligand', 'Receptor']

In [89]:
cell_LR_nodes

,gene,expression,identifier,category,cellgroup
0,CCL14,0.198610,CCL14_0_Ligand,Ligand,0
1,CCL14,0.402186,CCL14_1_Ligand,Ligand,1
2,CCL14,0.437958,CCL14_2_Ligand,Ligand,2
3,CCL14,0.704263,CCL14_3_Ligand,Ligand,3
4,CCL14,0.338172,CCL14_4_Ligand,Ligand,4
...,...,...,...,...,...
5870,B2M&FCGRT,2.222969,B2M&FCGRT_6_Receptor,Receptor,6
5871,ITGAV&ITGB1,2.222969,ITGAV&ITGB1_6_Ligand,Ligand,6
5872,ITGAV&ITGB5,2.222969,ITGAV&ITGB5_6_Ligand,Ligand,6
5873,ITGB1&ITGAV,2.222969,ITGB1&ITGAV_6_Ligand,Ligand,6


### complex 亚基计入表达后，是否应该删除单体表达 ###

In [90]:
LR_interactions

,Src,Dst,Sorce
0,OCLN,OCLN,cellcall
1,CCL26,CCR1,cellcall
2,CCL26,CCR3,cellcall
3,CCL26,CCR5,cellcall
4,CCL26,CX3CR1,cellcall
...,...,...,...
20228,CRIPTO,ACVR1B,sctensor
20229,CRIPTO,ACVR1C,sctensor
20230,CRIPTO,ACVR2B,sctensor
20231,CRIPTO,GPC1,sctensor


In [93]:
single_nodes = LR_interactions['Src'].unique().tolist() + LR_interactions['Dst'].unique().tolist()

In [97]:
single_nodes.__len__()

4759

In [96]:
list(set(single_nodes) & set(complex_nodes['subnit'])).__len__()

383

In [101]:
set(complex_nodes['subnit']).__len__()

447

In [92]:
complex_nodes

,subnit,complex,category
0,ACVRL1,ACVRL1&ACVR2A,Receptor
1,ACVR2A,ACVRL1&ACVR2A,Receptor
2,AMHR2,AMHR2&ACVR1,Receptor
3,ACVR1,AMHR2&ACVR1,Receptor
4,TGFBR1,TGFBR1&TGFBR2&TGFBR3,Receptor
...,...,...,...
1421,IL1B,IL1B&IL1R2&IL1RAP,Receptor
1422,IL1R2,IL1B&IL1R2&IL1RAP,Receptor
1423,IL1RAP,IL1B&IL1R2&IL1RAP,Receptor
1424,ITGAM,ITGAM&ITGB2,Ligand


In [103]:
complex_nodes.loc[complex_nodes['subnit'].isin(single_nodes)]

,subnit,complex,category
0,ACVRL1,ACVRL1&ACVR2A,Receptor
1,ACVR2A,ACVRL1&ACVR2A,Receptor
2,AMHR2,AMHR2&ACVR1,Receptor
3,ACVR1,AMHR2&ACVR1,Receptor
4,TGFBR1,TGFBR1&TGFBR2&TGFBR3,Receptor
...,...,...,...
1421,IL1B,IL1B&IL1R2&IL1RAP,Receptor
1422,IL1R2,IL1B&IL1R2&IL1RAP,Receptor
1423,IL1RAP,IL1B&IL1R2&IL1RAP,Receptor
1424,ITGAM,ITGAM&ITGB2,Ligand


In [100]:
LR_interactions.loc[LR_interactions['Src'] == 'ACVRL1']

,Src,Dst,Sorce
3583,ACVRL1,ACVR2A,cellinker
3637,ACVRL1,CDH5,cellinker
3640,ACVRL1,TGFBR1,cellinker
7988,ACVRL1,TGFB1,omnipath
7989,ACVRL1,TGFBR2,omnipath
10349,ACVRL1,ENG,omnipath
11489,ACVRL1,XIAP,omnipath


In [ ]:
def calculate_complex_expression(cell_LR_nodes, valid_complexes_dict, complex_nodes):
    complex_expr = {}
    for cellgroup in cell_LR_nodes['cellgroup'].unique():
        complex_expr[cellgroup] = {}
        cell_LR_nodes_cellgroup = cell_LR_nodes[cell_LR_nodes['cellgroup'] == cellgroup]
        for i in valid_complexes_dict[cellgroup]:
            complex_expr[cellgroup][i] = []
            for j in i.split('&'):
                complex_expr[cellgroup][i].append(cell_LR_nodes_cellgroup[cell_LR_nodes_cellgroup['gene'] == j]['expression'].values[0])
    complex_expr_df = []
    for i in complex_expr.keys():
        for j in complex_expr[i].keys():
            complex_expr_df.append([j, i, np.min(complex_expr[i][j])])
    complex_expr_df = pd.DataFrame(complex_expr_df, columns=['gene', 'cellgroup', 'expression'])
    category_complex = []
    for i in complex_expr_df.index:
        category_complex.append(complex_nodes.loc[complex_nodes['complex'] == complex_expr_df.iloc[i]['complex']]['category'].values[0])
    complex_expr_df['category'] = category_complex
    complex_expr_df['identifier'] = complex_expr_df['gene'] + '_' + complex_expr_df['cellgroup'] + '_' + complex_expr_df['category']
    return complex_expr_df
